In [1]:
# Code Based On: https://github.com/r4isstatic/csv-to-ttl/blob/master/uber.py
import csv
from rdflib import URIRef, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS, XSD
import uuid

In [2]:
bnode_g = Graph()

In [3]:
# namespaces
def create_namespace(graph, namespace, prefix):

    ns = Namespace(namespace)
    graph.namespace_manager.bind(prefix, namespace)
    
    return ns


hyp_namespace = create_namespace(bnode_g, "http://example.org/hypothesis_ontology/", 'hyp')
oa_namespace = create_namespace(bnode_g, "http://www.w3.org/ns/oa#", 'oa')
covidpr_namespace = create_namespace(bnode_g, "http://ns.inria.fr/covid19/property/", 'covidpr')
dct_namespace = create_namespace(bnode_g, "http://purl.org/dc/terms/", 'dct')
prov_namespace = create_namespace(bnode_g, 'http://www.w3.org/TR/prov-o/', 'prov')

# hyp_namespace = Namespace("http://example.org/hypothesis_ontology/")
# bnode_g.namespace_manager.bind('hyp', hyp_namespace)

# oa_namespace = Namespace("http://www.w3.org/ns/oa#")
# bnode_g.namespace_manager.bind('oa', oa_namespace)

# covidpr_namespace = Namespace("http://ns.inria.fr/covid19/property/")
# bnode_g.namespace_manager.bind("covidpr", covidpr_namespace)

# dct_namespace = Namespace("http://purl.org/dc/terms/")
# bnode_g.bind("dct", dct_namespace)

# prov_namespace = Namespace('http://www.w3.org/TR/prov-o/')
# bnode_g.bind("dct", dct_namespace)

In [4]:
ifile = open('./bnode_new1.csv')
reader = csv.reader(ifile)

In [5]:
rownum = 0
for row in reader:
    if rownum == 0: 
        pass
    else:
        unique_id = str(uuid.uuid4())
        unique_NE_uri = URIRef(hyp_namespace + unique_id + '#NamedEntity')
        bnode_g.add( (unique_NE_uri, oa_namespace.hasSource, hyp_namespace[row[3]]) )
        bnode_g.add( (unique_NE_uri, RDF.type, prov_namespace.Entity) )
        bnode_g.add( (unique_NE_uri, RDF.type, oa_namespace.Annotation) )
        bnode_g.add( (unique_NE_uri, oa_namespace.exact, Literal(row[1])) )
        bnode_g.add( (unique_NE_uri, oa_namespace.hasBody, URIRef(row[5])) )
        bnode_g.add( (unique_NE_uri, covidpr_namespace.confidence, Literal(row[4], datatype=XSD.decimal)) )
        subjects = row[2].split(',')
        for subj in subjects:
            bnode_g.add( (unique_NE_uri, dct_namespace.subject, Literal(subj)) )
    rownum += 1 

g = bnode_g.serialize('../Output-Graphs/bnode_test.ttl', format='turtle')

ifile.close()

In [6]:
import pprint
print(bnode_g.serialize(format='turtle'))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

